In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
from pathlib import Path

import pdblp
import numpy as np
import pandas as pd
from pandas.tseries.offsets import BDay

if (path := 'C:\\dev\\projects\\credit\\backend') not in sys.path:
    sys.path.append(path)

from api.blp import BlpQuery
import utils

In [ ]:
root_path = Path('/dev/projects/credit/backend/')
data_path = root_path / 'data'

### Bloomberg Connection

In [ ]:
name_mapping = {
    'INDEX_Z_SPREAD': 'z_spread',
    'CONTRBTD_ZSPREAD': 'z_spread',
    'PX_LAST': 'z_spread'
}

In [ ]:
dt_fmt = '%Y%m%d'
bq = BlpQuery(timeout=20000).start()

In [ ]:
indices_ref = pd.read_csv(data_path / 'indices_reference.csv')
indices_hist = pd.read_csv(data_path / 'indices_history.csv', parse_dates=['date'], dayfirst=True) 

In [ ]:
start_date = pd.Timestamp(2007, 1, 1)
end_date = pd.Timestamp.today()

In [ ]:
new_indices = indices_ref.loc[~indices_ref['ticker'].isin(indices_hist['security'].unique())]
# new_indices = new_indices.loc[new_indices['fields'] != "['CONTRBTD_ZSPREAD']", :]

In [ ]:
new_indices

Empty DataFrame
Columns: [ticker, name, fields, scale]
Index: []

In [ ]:
if not new_indices.empty:
    ress = {}

    for index, row in new_indices.iterrows():
        print(f"downloading data for {row['ticker']}")
        res = bq.bdh(
            securities=[row['ticker']],
            fields=eval(row['fields']),
            start_date=start_date.strftime(dt_fmt), 
            end_date=end_date.strftime(dt_fmt)
        )
        res = res.rename(name_mapping, axis=1)
        res['z_spread'] = res['z_spread'] / row['scale']
        ress[row['ticker']] = res 
    new_hist = pd.concat(ress.values(), keys=None)
    indices_hist = pd.concat([indices_hist, new_hist])\
                     .sort_values(['security', 'date'])\
                     .reset_index(drop=True)
    indices_hist.to_csv(data_path / 'indices_history.csv', index=False)

downloading data for I03139EU Index
downloading data for SUBFIN CDSI GEN 5Y Corp
downloading data for SNRFIN CDSI GEN 5Y Corp
downloading data for CDX IG CDSI GEN 5Y Corp
downloading data for CDX HY CDSI GEN 5Y SPRD Corp
downloading data for BLQCTRUU Index
downloading data for BHYOTRUU Index


In [ ]:
# indices_hist.loc[indices_hist['security'] == 'BUS3TRUU Index']

date        security  z_spread
11640 2007-01-02  BUS3TRUU Index  0.665987
11641 2007-01-03  BUS3TRUU Index  0.665461
11642 2007-01-04  BUS3TRUU Index  0.664393
11643 2007-01-05  BUS3TRUU Index  0.658689
11644 2007-01-08  BUS3TRUU Index  0.658635
...          ...             ...       ...
15494 2022-05-18  BUS3TRUU Index  1.182188
15495 2022-05-19  BUS3TRUU Index  1.183101
15496 2022-05-20  BUS3TRUU Index  1.200134
15497 2022-05-23  BUS3TRUU Index  1.186637
15498 2022-05-24  BUS3TRUU Index  1.173695

[3859 rows x 3 columns]